## Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Navigating to Data Folder

In [1]:

data_folder = '/Users/gautam.v/Codes/AIISC/data'
%cd $data_folder

/Users/gautam.v/Codes/AIISC/data


## Installations

In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy triton scipy ftfy spacy==3.5.0
%pip install -q xformers==0.0.16rc425
%pip install daam==0.0.11

## Importing Libraries

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, DPMSolverMultistepScheduler
import mediapy as media
import torch
from diffusers import StableDiffusionPipeline
import random
import pandas as pd
import numpy as np
import os
from transformers import CLIPProcessor, CLIPModel
import daam
import time
from torch import autocast

## Choosing the Model

In [ ]:
diffusion_model_id = "stabilityai/stable-diffusion-2-1-base"
# diffusion_model_id = "stabilityai/stable-diffusion-2-1"
# diffusion_model_id = "dreamlike-art/dreamlike-photoreal-2.0"

## Choosing the Scheduler

In [ ]:
scheduler = None
# scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")

## Hyperparameters and Model Parameters

In [ ]:
# Decide the device
device = "cuda" if torch.cuda.is_available() else "cpu"
if diffusion_model_id.startswith("stabilityai/"):
  model_revision = "fp16"
else:
  model_revision = None

## Defining Image Size Parameters

In [ ]:
if diffusion_model_id.endswith('-base'):
  image_length = 512
else:
  image_length = 768

## Actual Pipeline

#### Read the data

In [ ]:
data_path = 'all_claims.xlsx' # Replace with the actual path to the data
df = pd.read_excel(data_path)

## Clip Rating

In [ ]:
def get_clip_scores(claim, lst,processor,model):
  inputs = processor(text=claim, images=lst, return_tensors="pt", padding=True)
  outputs = model(**inputs)
  logits_per_image = outputs.logits_per_image # this is the image-text similarity score
  best_image = max(logits_per_image)
  arg_max = torch.argmax(logits_per_image)
  return logits_per_image, best_image, arg_max

#### Auxillary and IO Functions


In [ ]:
def write_file(filename, intval):
    with open(filename, 'w') as fp:
        fp.write(str(intval))

def read_file(filename):
    with open(filename) as fp:
        return fp.read()
    
def set_seed(seed):
  gen = torch.Generator(device=device)
  random.seed(seed)
  np.random.seed(seed)
  torch.cuda.manual_seed(seed)
  torch.manual_seed(seed)
  return gen

## Generation for N Claims

#### Important Parameters

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(diffusion_model_id)
pipe = pipe.to(device)
# prompt = 'Deepika Padukone buying liquor in Mumbai' # provide the prompt , iterate over the list of claims in fever and run the loop
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
# N is the number of claims
N = 5
# num_images is the number of images to be generated per claim
num_images = 8
# Base path of where the images and logs will be stored
base_path = "/Users/gautam.v/Codes/DeHate/generated_images"

#### Decide if Negative Prompts Need to be Omitted or not

In [ ]:
remove_safety = False # can be changed to True when and if needed
if remove_safety:
  negative_prompt = None
  pipe.safety_checker = None
else:
  negative_prompt = "nude, naked"

#### Actual Pipeline

In [ ]:
for claim_idx in range(N):
  assert "claim" in df.columns, "The column name should be claim"
  claim_folder = os.path.join(base_path, f"Claim_idx_{claim_idx}")
  if not os.path.exists(claim_folder):
    os.makedirs(claim_folder)
  claim_text = df.claim[claim_idx]
  images_list = []
  seed_time = int(time.time())
  gen = set_seed(seed_time)  # for reproducibility
  with daam.trace(pipe) as tc:
    with autocast(device):
      for image_idx in range(1,num_images+1):
        image = pipe(
        claim_text,
        height = image_length,
        width = image_length,
        num_inference_steps = 25, # this is just for trial. change to >=500 for good results
        guidance_scale = 9,
        negative_prompt = negative_prompt,
        generator = gen
        )['images'][0]
        images_list.append(image)
        image_name = f"Image_no_{image_idx}.png"
        image_path = os.path.join(claim_folder, image_name)
        image.save(image_path)
        daam_path = os.path.join(claim_folder, f"daam_{image_idx}")
        exp = tc.to_experiment(daam_path)
        exp.save()  # experiment-dir now contains all the data and heat maps
  logits_per_image, best_image, arg_max = get_clip_scores(claim_text, images_list,processor,model)
  logging_file_name = os.path.join(claim_folder, "best_image_log.txt")
  logging_text = f"All CLIP Scores: {logits_per_image} Best Image: image_{int(arg_max)} with CLIP score of {best_image}"
  write_file(logging_file_name,logging_text)

  